### Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
user_question = "I just discovered the course. Can I still join it?"
embedding_vector = embedding_model.encode(user_question)

In [3]:
first_value = embedding_vector[0]
print(f"The first value in the vector is: {first_value:.3f}")

The first value in the vector is: 0.078


### Q2. Prepare the documents and create the embeddings

In [4]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [5]:
filtered_documents = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']

print(f"Number of filtered documents: {len(filtered_documents)}")

Number of filtered documents: 375


In [6]:
import numpy as np
from tqdm.auto import tqdm 

# Create embeddings for the filtered documents
embeddings = []
for doc in tqdm(filtered_documents):
    qa_text = f'{doc["question"]} {doc["text"]}'
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

# Convert the list of embeddings to a numpy array
X = np.array(embeddings)

# Print the shape of X
print(f"Shape of X: {X.shape}")

  0%|          | 0/375 [00:00<?, ?it/s]

100%|██████████| 375/375 [01:06<00:00,  5.66it/s]

Shape of X: (375, 768)


### Q3. Search

In [7]:
v = embedding_vector

scores = X.dot(v)

print(f"The highest score in the results is: {max(scores):.2f}")

The highest score in the results is: 0.65


In [8]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argpartition(-scores, num_results)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

### Q4. Hit-rate for our search engine

In [9]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [10]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [11]:
def search_function(q): 
    v_query = embedding_model.encode(q['question'])
    return search_engine.search(v_query, num_results=5)

In [12]:
evaluation_result = evaluate(ground_truth, search_function)

  0%|          | 2/1830 [00:00<01:34, 19.35it/s]

100%|██████████| 1830/1830 [01:33<00:00, 19.60it/s]


In [13]:
# Print the evaluation result
print(f"Hit rate: {evaluation_result['hit_rate']:.4f}")

Hit rate: 0.9399


### Q5. Indexing with Elasticsearch

In [17]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
            "embedding": {"type": "dense_vector", "dims": 768}
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [42]:
from elasticsearch.helpers import bulk

# Prepare documents for bulk indexing
def generate_actions(documents, embeddings):
    for doc, emb in zip(documents, embeddings):
        yield {
            "_index": index_name,
            "id": doc['id'],  # Ensure the document has a unique ID
            "_source": {
                "text": doc['text'],
                "question": doc['question'],
                "embedding": emb.tolist()  # Convert numpy array to list
            }
        }

# Index the documents with embeddings
actions = generate_actions(filtered_documents, X)
bulk(es_client, actions)

(375, [])

In [43]:
# Example query embedding function
query = "I just discovered the course. Can I still join it?"
v_query = embedding_model.encode(query).tolist()  # Convert numpy array to list

# Perform the search
search_query = {
    "query": {
        "script_score": {
            "query": {
                "match_all": {}
            },
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                "params": {
                    "query_vector": v_query
                }
            }
        }
    },
    "_source": ["text", "question"]
}

response = es_client.search(index=index_name, body=search_query)
top_hit = response['hits']['hits'][0]
print(f"ID of the document with the highest score: {top_hit['_id']}")
print(f"Question of the document: {top_hit['_source']['question']}")
print(f"Text of the document: {top_hit['_source']['text']}")


ID of the document with the highest score: A6nysJABXTX41oljXCU5
Question of the document: The course has already started. Can I still join it?
Text of the document: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.


### Q6. Hit-rate for Elasticsearch

In [48]:
# Define the search function using Elasticsearch
def search_function(q):
    v_query = embedding_model.encode(q['question']).tolist()  # Convert numpy array to list
    search_query = {
        "size": 5,
        "query": {
            "script_score": {
                "query": {
                    "match_all": {}
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {
                        "query_vector": v_query
                    }
                }
            }
        },
         "_source": ["text", "question"] 
    }
    response = es_client.search(index=index_name, body=search_query)
    return response['hits']['hits']


In [49]:
def evaluate_id(ground_truth, search_function):
    relevance_total = []
    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['_id'] == doc_id for d in results]  # Use _id to check relevance
        relevance_total.append(relevance)
    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [50]:
# Evaluate the search engine with Elasticsearch
evaluation_result = evaluate_id(ground_truth, search_function)

# Print the evaluation result
print(f"Hit rate: {evaluation_result['hit_rate']}")


100%|██████████| 1830/1830 [01:30<00:00, 20.18it/s]

Hit rate: 0.7874316939890711
